In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, AdamW
import numpy as np

In [2]:
train_df = pd.read_csv('C:/Users/brist/Downloads/goemotions_1.csv')

In [3]:
train_df.head()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
# Define emotion labels
emotion_labels = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity',
    'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
    'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
    'remorse', 'sadness', 'surprise', 'neutral'
]

In [5]:
# Convert pandas to HF Dataset
dataset = Dataset.from_pandas(train_df)

In [6]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(examples):
    encodings = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)
    labels = []
    for i in range(len(examples['text'])):
        label_vector = [examples[label][i] for label in emotion_labels]
        labels.append(label_vector)
    encodings['labels'] = labels
    return encodings

dataset = dataset.map(preprocess, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


C:\Users\brist\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

In [8]:
# Split locally since we only have train.csv loaded
split = dataset.train_test_split(test_size=0.2, seed=42)
train_ds = split['train']
val_ds = split['test']

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16)


In [9]:
from transformers import BertConfig

num_labels = len(emotion_labels)
config = BertConfig.from_pretrained(model_name, num_labels=num_labels)
model = BertForSequenceClassification.from_pretrained(model_name, config=config)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [10]:
from tqdm import tqdm
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)

model.train()
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    loop = tqdm(train_loader)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].float().to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        loop.set_description(f'Epoch {epoch + 1}')
        loop.set_postfix(loss=loss.item())


C:\Users\brist\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3


Epoch 1: 100%|██████████| 3500/3500 [9:54:35<00:00, 10.19s/it, loss=0.118]      


Epoch 2/3


Epoch 2: 100%|██████████| 3500/3500 [9:33:26<00:00,  9.83s/it, loss=0.105]       


Epoch 3/3


Epoch 3: 100%|██████████| 3500/3500 [6:31:06<00:00,  6.70s/it, loss=0.139]   


In [11]:
from sklearn.metrics import multilabel_confusion_matrix

model.eval()
threshold = 0.5

true_labels = []
pred_labels = []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        probs = torch.sigmoid(logits)
        preds = (probs > threshold).int().cpu().numpy()

        true_labels.append(labels.cpu().numpy())
        pred_labels.append(preds)

true_labels = np.vstack(true_labels)
pred_labels = np.vstack(pred_labels)

# Exact match accuracy (all labels must match)
accuracy = np.mean(np.all(true_labels == pred_labels, axis=1))
print(f"Exact Match Accuracy: {accuracy:.4f}")

# Confusion matrix for each label
mcm = multilabel_confusion_matrix(true_labels, pred_labels)
for i, label in enumerate(emotion_labels):
    tn, fp, fn, tp = mcm[i].ravel()
    print(f"{label}: TP={tp}, FP={fp}, FN={fn}, TN={tn}")


Exact Match Accuracy: 0.2251
admiration: TP=584, FP=451, FN=539, TN=12426
amusement: TP=385, FP=308, FN=216, TN=13091
anger: TP=65, FP=62, FN=429, TN=13444
annoyance: TP=8, FP=15, FN=879, TN=13098
approval: TP=95, FP=132, FN=1100, TN=12673
caring: TP=66, FP=126, FN=325, TN=13483
confusion: TP=66, FP=53, FN=454, TN=13427
curiosity: TP=156, FP=229, FN=498, TN=13117
desire: TP=34, FP=39, FN=194, TN=13733
disappointment: TP=27, FP=16, FN=519, TN=13438
disapproval: TP=103, FP=156, FN=676, TN=13065
disgust: TP=25, FP=10, FN=332, TN=13633
embarrassment: TP=6, FP=1, FN=152, TN=13841
excitement: TP=41, FP=41, FN=357, TN=13561
fear: TP=61, FP=42, FN=136, TN=13761
gratitude: TP=576, FP=79, FN=197, TN=13148
grief: TP=0, FP=0, FN=41, TN=13959
joy: TP=141, FP=172, FN=361, TN=13326
love: TP=289, FP=157, FN=253, TN=13301
nervousness: TP=0, FP=0, FN=115, TN=13885
optimism: TP=141, FP=93, FN=431, TN=13335
pride: TP=0, FP=0, FN=98, TN=13902
realization: TP=17, FP=9, FN=585, TN=13389
relief: TP=0, FP=0, F

In [12]:
import gradio as gr

def predict_emotions(text):
    inputs = tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.sigmoid(logits).cpu().numpy()[0]
    result = {emotion_labels[i]: float(probs[i]) for i in range(len(emotion_labels))}
    return result

iface = gr.Interface(fn=predict_emotions, inputs="text", outputs="label", title="GoEmotions BERT Model")
iface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
